In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
motion = np.genfromtxt('../../../dataset4_wbc/live_extraction/step_0.06/gt/fwd_only/accelerations.csv', delimiter=',', dtype=np.float64)
contacts = np.genfromtxt('../../../dataset4_wbc/live_extraction/step_0.06/gt/fwd_only/accelerations.csv', delimiter=',', usecols=(57, 58, 59, 60), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  4281


In [6]:
motion.shape

(4281, 61)

In [7]:
i = 0
accelerations_idx = []
while i < (len(motion) - 1):
    same_sign = np.all(np.sign(motion[i, 1:4]) == np.sign(motion[i+1, 1:4]))
    if (motion[i, 1] != motion[i+1, 1] or motion[i, 2] != motion[i+1, 2] or motion[i, 3] != motion[i+1, 3]) and (motion[i, 1] >= 0.0 and  motion[i+1, 1] >= 0.0):
        #print(motion[i, 1], motion[i+1, 1])
        #print(motion[i, 1] != motion[i+1, 1])
        accelerations_idx.append(i)
        accelerations_idx.append(i+1)
    
    i += 1

accelerations_idx = np.array(accelerations_idx)

In [8]:
accelerations_idx.shape

(1930,)

## Height-Force plots

In [9]:
def get_specific_cmd(dataset, fwd, side, rot):
    if abs(fwd): 
        return np.where(dataset[:, 1] > 0)[0] if fwd > 0 else np.where(dataset[:, 1] < 0)[0]
    if abs(side): 
        return np.where(dataset[:, 2] > 0)[0] if side > 0 else np.where(dataset[:, 2] < 0)[0]
    if abs(rot): 
        return np.where(dataset[:, 3] > 0)[0] if rot > 0 else np.where(dataset[:, 3] < 0)[0]

In [10]:
def get_swinging_motions(dataset, height=1):
    # rf min height (i.e swining motion)
    fl_min_height = np.where(dataset[:, 10] < height)[0]
    fr_min_height = np.where(dataset[:, 13] < height)[0]
    rl_min_height = np.where(dataset[:, 16] < height)[0]
    rr_min_height = np.where(dataset[:, 19] < height)[0]
            
    return fl_min_height,fr_min_height,rl_min_height,rr_min_height

# Dataset Preparation

In [15]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [12]:
def euler_from_quaternion(x, y, z, w):
    """
    Convert a quaternion into euler angles (roll, pitch, yaw)
    roll is rotation around x in radians (counterclockwise)
    pitch is rotation around y in radians (counterclockwise)
    yaw is rotation around z in radians (counterclockwise)
    """
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll_x = math.atan2(t0, t1)
     
    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch_y = math.asin(t2)
     
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw_z = math.atan2(t3, t4)
     
    return roll_x, pitch_y, yaw_z # in radians

In [20]:
def create_com_dataset_absolute(dataset, footsteps, motion=None, debug=False):
    inputs = []
    labels = []
    velocity_x = 0
    velocity_y = 0
    velocity_yaw = 0
    
    for idx, footstep in enumerate(footsteps):
        if idx > 0:
            # Compute time difference between
            # current and next footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])
            
            if time_difference < 0.5:
                fl_moving = dataset[footsteps[idx], 8] > dataset[footsteps[idx-1], 8]
                fr_moving = dataset[footsteps[idx], 11] > dataset[footsteps[idx-1], 11]
                rl_moving = dataset[footsteps[idx], 14] > dataset[footsteps[idx-1], 14]
                rr_moving = dataset[footsteps[idx], 17] > dataset[footsteps[idx-1], 17]
                
                if debug:
                    print(idx, height_peeks_in_between)
                    print("Time difference: ", time_difference)
                    print("Footstep timestamp: ", dataset[footstep, 0])
                    print(f'FL moving: {fl_moving}. FR moving: {fr_moving}')
                    print(f'RL moving: {rl_moving}. RR moving: {rr_moving}')

                if (not fr_moving != fl_moving or not fr_moving == rl_moving or not fl_moving == rr_moving):
                    #print("Exception")
                    #print("Time difference: ", time_difference)
                    #print("Footstep timestamp: ", dataset[footstep, 0])
                    #print(f'fl moving: {fl_moving}. fr moving: {fr_moving}')
                    #print(f'rl moving: {rl_moving}. rr moving: {rr_moving}')
                    continue
                
                assert fr_moving == rl_moving
                assert rr_moving == fl_moving
                assert fr_moving != fl_moving
                
                # Get euler angles from odom to base rotation
                _, _, curr_yaw = euler_from_quaternion(dataset[footsteps[idx-1], 47], dataset[footsteps[idx-1], 48], dataset[footsteps[idx-1], 49], dataset[footsteps[idx-1], 50])
                _, _, next_yaw = euler_from_quaternion(dataset[footsteps[idx], 47], dataset[footsteps[idx], 48], dataset[footsteps[idx], 49], dataset[footsteps[idx], 50])

                # Compute rotation matrix
                c_curr, s_curr = np.cos(curr_yaw), np.sin(curr_yaw)
                c_next, s_next = np.cos(next_yaw), np.sin(next_yaw)
                R_curr = np.array(((c_curr, -s_curr), (s_curr, c_curr)))
                R_next = np.array(((c_next, -s_next), (s_next, c_next)))
                
                # Odom base vectors
                curr_base = np.array([[dataset[footsteps[idx-1], 44], dataset[footsteps[idx-1], 45]]])
                next_base = np.array([[dataset[footsteps[idx], 44], dataset[footsteps[idx], 45]]])
                
                # Local base vectors
                local_curr_base = R_curr.T @ curr_base.T
                local_next_base = R_curr.T @ next_base.T
                base_displacement = local_next_base - local_curr_base
                
                if (dataset[footsteps[idx], 44] - dataset[footsteps[idx-1], 44]) > 0:
                    inputs.append(dataset[footsteps[idx-1], 1:4].tolist() +
                                  dataset[footsteps[idx-1], 51:57].tolist() +
                                  dataset[footsteps[idx-1], 8:20].tolist() + 
                                  [fl_moving, fr_moving, rl_moving, rr_moving] +
                                  dataset[footsteps[idx], 1:4].tolist())
                
                    # Labels
                    labels.append([dataset[footsteps[idx], 44] - dataset[footsteps[idx-1], 44],
                                   dataset[footsteps[idx], 45] - dataset[footsteps[idx-1], 45]])
                
    # Convert to numpy array
    inputs_numpy = np.array(inputs, dtype=object)
    labels_numpy = np.array(labels, dtype=object)
    
    return inputs_numpy, labels_numpy

# Stack datasets
X_motion_CoM_acceleration, Y_motion_CoM_acceleration = create_com_dataset_absolute(motion, [x for x in range(len(motion))])
print(X_motion_CoM_acceleration.shape)
print(Y_motion_CoM_acceleration.shape)

(4084, 28)
(4084, 2)


In [21]:
%store X_motion_CoM_acceleration
%store Y_motion_CoM_acceleration

Stored 'X_motion_CoM_acceleration' (ndarray)
Stored 'Y_motion_CoM_acceleration' (ndarray)


In [22]:
min(Y_motion_CoM_acceleration[:, 0])

0.0009849669999937305

In [23]:
max(Y_motion_CoM_acceleration[:, 0])

0.09256132800001637

In [24]:
min(Y_motion_CoM_acceleration[:, 1])

-0.040583139549999814

In [25]:
max(Y_motion_CoM_acceleration[:, 1])

0.04212986456000012